In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
% matplotlib inline

# What's TFRecord

TFRecord是tensorflow所使用的一种二进制文件格式。二进制文件的优点在于可以带有一些头部信息，让tensorflow可以知道整个文件的结构，快速地读取数据。

TFRecord将数据组织成Example，每个Example表示一条记录。每个Example由tf.train.Feature组成，多个feature被表示成tf.train.Features.

tf.train.Feature可以储存三种不同类型的数据： bytes,float,int64

但是有些Feature是数组形式的，比如一张图片，为了简化和抽象，tf.train.Feature的三种格式都需要以列表的形式储存：bytes_list,float_list和int64_list。对于scalar类型的数据和高维数据都需要转换成为列表形式，即1维数据。




假设我们有一张大小为2*2的图片，[[1.0,2.0],[3.0,4.0]],这张图片的label是4，我们尝试将这张图片写入TFRecord中
我们首先要将图片变成1维的列表，实际上我们有两种方式，
 1.保存图片的float值
 2.将图片转换成bytes

In [3]:
image = np.array([[1.0,2.0],[3.0,4.0]])
image_float = image.flatten()

image_bytes = image.tobytes()

In [4]:
tfrecord_bytes_filename = "test_bytes.tfrecord"
writer = tf.python_io.TFRecordWriter(tfrecord_bytes_filename)

my_example = tf.train.Example(features=tf.train.Features(feature={
    'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[4])),
    'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[image_bytes]))
}))
writer.write(my_example.SerializeToString())
writer.close()

In [5]:
tfrecord_float_filename = "test_float.tfrecord"
writer = tf.python_io.TFRecordWriter(tfrecord_float_filename)

my_example = tf.train.Example(features=tf.train.Features(feature={
    'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[4])),
    'image': tf.train.Feature(float_list =tf.train.FloatList(value=image_float))
}))
writer.write(my_example.SerializeToString())
writer.close()

# Read TFRecord

读取TFRecord的时候需要使用tf.TFRecordReader来读取TFRecord文件，但是对于每个example需要使用parse_single_example函数来对二进制数据进行解析，将二进制重新转换为int64,float以及bytes等格式。parse_single_example函数需要传入一个shape参数，表明将解析出来的数据list进行reshape。

其中，[]表示标量，[5]表示长度为5的数组，[28,28,3]表示对应大小的3维数组。

In [6]:
sess = tf.InteractiveSession()

## 分别读取float和bytes格式的TFRecords

In [13]:
#tfrecord_float_filename = "test_float.tfrecord"
reader = tf.TFRecordReader()

float_filename_queue = tf.train.string_input_producer([tfrecord_float_filename])
float_key,float_value = reader.read(float_filename_queue)

bytes_filename_queue = tf.train.string_input_producer([tfrecord_bytes_filename])
bytes_key,bytes_value = reader.read(bytes_filename_queue)

float_example = tf.parse_single_example(float_value,
                    features={"image":tf.FixedLenFeature([2,2],tf.float32),
                              "label":tf.FixedLenFeature([],tf.int64)})

bytes_example = tf.parse_single_example(bytes_value,
                    features={"image":tf.FixedLenFeature([],tf.string),
                              "label":tf.FixedLenFeature([],tf.int64)})
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord)


In [14]:
float_example_val,bytes_example_val = sess.run([float_example,bytes_example])
print(float_example_val)
print(bytes_example_val)

{'image': array([[ 1.,  2.],
       [ 3.,  4.]], dtype=float32), 'label': 4}
{'image': b'\x00\x00\x00\x00\x00\x00\xf0?\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@', 'label': 4}


我们可以看出，如果我们之前将图片表示成bytes类型，那么解析出来也只有一个bytys字符串，此时我们可以使用decode_raw将bytes字符串转换成其他类型数据。

In [20]:
image_reback = tf.decode_raw(bytes_example_val["image"],tf.float64)
image_reback = tf.reshape(image_reback,[2,2])

In [21]:
image_reback_val = sess.run([image_reback])
print(image_reback_val)

[array([[ 1.,  2.],
       [ 3.,  4.]])]


In [22]:
my_example = tf.train.Example(features=tf.train.Features(feature={
    'index_0': tf.train.Feature(int64_list=tf.train.Int64List(value=[0, 1, 2])),
    'index_1': tf.train.Feature(int64_list=tf.train.Int64List(value=[5, 1, 4])),
    'values': tf.train.Feature(int64_list=tf.train.Int64List(value=[7, 5, 9]))
}))
my_example_str = my_example.SerializeToString()

In [26]:
my_example_features = {'sparse': tf.SparseFeature(index_key=['index_0', 'index_1'],
                                                  value_key='values',
                                                  dtype=tf.int64,
                                                  size=[4, 6])}
serialized = tf.placeholder(tf.string)
parsed = tf.parse_single_example(serialized, features=my_example_features)
sess.run(parsed, feed_dict={serialized: my_example_str})

{'sparse': SparseTensorValue(indices=array([[0, 5],
       [1, 1],
       [2, 4]], dtype=int64), values=array([7, 5, 9], dtype=int64), dense_shape=array([4, 6], dtype=int64))}